# 1 环境需求

transformer
qwen-vl-utils

In [1]:
from transformers import Qwen2_5_VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

MODEL_DIR = r"D:\Models\Qwen\Qwen2.5-VL-3B-Instruct" # 模型路径

# 2 加载模型

## 2.1 准备模型

可以讲模型下载到本地，也可以直接从 Hugging Face 上加载。

```python
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)
```
这样会自动下载模型到本地缓存目录（通常是 `~/.cache/huggingface/transformers`，不做特殊设置在 C 盘，不推荐）。

如果想指定模型存放目录，可以先把模型下载到本地，然后再从本地加载：

```python
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    r"D:\Models\Qwen\Qwen2.5-VL-3B-Instruct", torch_dtype="auto", device_map="auto"
)
```

In [2]:
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    MODEL_DIR, dtype="auto", device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## 2.2 参数设置

```python
model = Qwen2_5_VLForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path,  # 模型名称或路径
    cache_dir=None,                 # 模型缓存目录
    revision=None,                  # 模型版本（分支/tag/commit）
    torch_dtype=None,               # 加载时的精度
    device_map=None,                # 模型放在哪些设备上
    low_cpu_mem_usage=False,        # 大模型加载时常用，减少 CPU 内存占用
    local_files_only=False,         # 只用本地文件，不联网下载
)
```

1. **`pretrained_model_name_or_path`**

   * 必填，模型名称或路径。
   * 例子：`"bert-base-uncased"` 或 `"./my_model_checkpoint/"`

2. **`cache_dir`**

   * 设置下载模型的缓存路径。
   * 避免每次都从 Hugging Face Hub 下载。

3. **`revision`**

   * 指定模型的版本（分支/tag/commit）。
   * 例子：`revision="v1.0.0"`

4. **`torch_dtype`**

   * 指定加载时的精度。
   * 常用：`torch.float16`（省显存）、`"auto"`（自动选择）。

5. **`device_map`**

   * 指定模型放在哪些设备上。
   * 常用：`"auto"`（自动把大模型切分到多个 GPU/CPU）。

6. **`low_cpu_mem_usage`**

   * 大模型加载时常用，减少 CPU 内存占用。
   * 常用：`True`

7. **`local_files_only`**

   * 只用本地文件，不联网下载。
   * 常用：`True`（离线环境）

对于 Qwen2.5vl，官方推荐如果做多图或者视频等长序列任务，可以使用`attn_implementation="flash_attention_2"`将注意力机制替换为 Flash Attention 2，推理更快、更省显存。

## 2.3 准备 processor

```python
processor = AutoProcessor.from_pretrained("Qwen/Qwen2.5-VL-3B-Instruct")
```

在使用 Hugging Face 加载模型时，需要同时加载对应的 processor。processor 负责将输入数据（文本、图像等）转换为模型可以处理的格式。

Qwen官方提供了一个能够同时处理文本、图像和视频的 processor。

同理，如果已经下好了模型可以直接加载本地的：

```python
processor = AutoProcessor.from_pretrained(r"D:\Models\Qwen\Qwen2.5-VL-3B-Instruct")
```

In [3]:
processor = AutoProcessor.from_pretrained(MODEL_DIR)

The image processor of type `Qwen2VLImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. Note that this behavior will be extended to all models in a future release.


# 3 准备数据

## 3.1 加载数据

构建一个列表，格式为：
```python
messages = [
    {...},   # 第一轮消息
    {...},   # 第二轮消息
    {...}    # 第三轮消息
]
```
每一轮消息是一个字典，一般包含：
- `"role"`：角色，通常是 `"user"`（用户）、 `"assistant"`（助手）、 `"system"`（系统），其中：
    - `"user"`：用户输入
    - `"assistant"`：模型输出
    - `"system"`：系统提示，通常在对话开始时使用，提供背景信息或指导
- `"content"`：消息内容，可以是文本、图像或视频等，context 也是一个列表，格式为：
```python
"context": [
    {"type": "text", "text": "文本内容"},
    {"type": "image", "image": image},  # image 是 PIL.Image 对象
    {"type": "video", "video": video_path}  # video_path 是视频文件路径
]
```

In [4]:
messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
                "image": "images/demo.jpeg",
            },
            {"type": "text", "text": "What is on the left side of the picture?"},
        ],
    }
]

## 3.2 处理数据

加载好的数据不能直接给模型，都是要经过处理的。

1. 文本处理：
    ```python
    text = processor.apply_chat_template(
        messages, 
        tokenize=False,                   # 是否进行分词，处理为 token id，这里不需要，因为后面有 processor(...) 会统一进行处理
        add_generation_prompt=True        # 是否在最后加上助手的提示符（类似于下面格式中的 <|assistant|>），告诉模型现在该它说话了
    )
    ```
    apply_chat_template 会将 messages 中的文本进行处理为模型所需的聊天格式，一般来讲模型都会有聊天格式，类似这样：
    ```
    <|system|>
    You are a helpful assistant.

    <|user|>
    Describe this image.

    <|assistant|>
    ```
    这样有两个好处：
    1. 让模型更好地理解对话的结构和角色
    2. 如果有多轮对话，它会按顺序拼好，保持上下文。


2. 图像和视频处理：
    ```python
    image_inputs, video_inputs = process_vision_info(messages)
    ```
    从 messages 里把视觉模态的内容（图片、视频）抽取出来，并转成后续 processor(...) 可以直接处理的标准格式。


3. 拼接

```python
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,             # 是否补 <pad> token，对齐到同一长度
    return_tensors="pt",      # 返回 PyTorch 张量，"tf" 返回 TensorFlow 张量，"np" 返回 NumPy 数组
)
```
这样 inputs 就是一个字典，里面包含了模型需要的所有输入。


4. 移动到设备

```python
inputs = inputs.to("cuda")
```

In [5]:
text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

# 4 推理

## 4.1 生成

```python
generated_ids = model.generate(
    **inputs,                   # 解包 inputs 字典，传给模型
    max_new_tokens=128          # 生成的最大长度
    )
```
这一步会把预处理好的 inputs 喂进模型，模型会进入解码循环（generation loop），生成新的 token id，直到达到 `max_new_tokens` 或者遇到结束符，返回生成的 token 序列。

## 4.2 解码

```python
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
```
这一步是把生成的 token id 序列进行裁剪，去掉输入部分，只保留模型新生成的部分。

## 4.3 转文本

```python
output_text = processor.batch_decode(
    generated_ids_trimmed, 
    skip_special_tokens=True,                 # 是否去除特殊符号，如 <pad>、<eos> 等
    clean_up_tokenization_spaces=False        # 是否清理多余的空格
)
```
这一步是把生成的 token id 转回文本字符串。

In [6]:
generated_ids = model.generate(**inputs, max_new_tokens=128)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

# 5 输出

```python
print(output_text)
```
打印模型输出。

In [7]:
print(output_text)

# 这部分不用管，看完后面多轮对话后再回来看
messages.append({
    "role": "assistant", 
    "content": [
        {
            "type": "text", 
            "text": output_text
        }
    ],
})

['On the left side of the picture, there is an ocean with waves crashing onto the shore. The sky above the ocean appears to be clear and bright, suggesting it might be a sunny day.']


# 6 拓展

## 6.1 生成参数值

```python
generated_ids = model.generate(
    **inputs,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,   # 设置想要的温度
    top_p=0.9          # 可选，常用组合
)
```
可以在 generate 时设置 temperature 和 top_p。

## 6.2 多轮对话

多轮对话的核心就是每一轮对话始终带上历史 messages，方法就是每一次新对话都将消息`append`进 messages 中即可。

这里需要说明一点，因为我们是使用`append`方法来加入新消息的，所以第一轮传入的图片模型第二轮第三轮等后面都能看见，如果是对同一张图片无需传多次。

后面继续前面的操作即可。

当模型回答完后也要将模型的回答`append`到 messages 中。

In [10]:
messages.append({
    "role": "user",
    "content": [
        {
            "type": "text", 
            "text": "What are on the right and in the middle of the picture?"
        }
    ],
})

text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt",
)
inputs = inputs.to("cuda")

generated_ids = model.generate(
    **inputs,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.7,   # 设置想要的温度
    top_p=0.9          # 可选，常用组合
)
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]
output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)

print(output_text)

messages.append({
    "role": "assistant", 
    "content": [
        {
            "type": "text", 
            "text": output_text
        }
    ],
})

['["[\'On the right side of the picture, there is a woman sitting on the beach with her dog. She is smiling and appears to be interacting with the dog, who is also sitting on the sand and looking at her.\']"]']
